<h1> Recommendations based on Nutrition preferences</h1>

<h3> 1. Import libraries and dataset </h3>

In [4]:
import pandas as pd
import numpy as np
import os, io
import sklearn
from sklearn.preprocessing import normalize
from scipy.spatial.distance import cosine, euclidean, hamming
from time import time
from sklearn.metrics import mean_squared_error

In [5]:
interactions_train  = pd.read_csv('interactions_train_new.csv',index_col=1)# read csv into ratings_df dataframe
interactions_train.drop(columns=['date','u','i','rating'],inplace=True)
interactions_train.index.names = ['id']
interactions_train.shape

(698901, 2)

In [6]:
interactions_test  = pd.read_csv('interactions_test_new.csv')# read csv into ratings_df dataframe
interactions_test.drop(columns=['date','u','i','rating'],inplace=True)
interactions_test.head(3)

,user_id,recipe_id,without_0_rating
0,8937,44551,4.0
1,56680,126118,4.0
2,349752,219596,5.0


In [7]:
raw_recipes  = pd.read_csv('RAW_recipes.csv',index_col=1)# read csv into ratings_df dataframe
mask1=raw_recipes.index.isin(interactions_train.index)
mask2=raw_recipes.index.isin(interactions_test.recipe_id)
raw_recipes=raw_recipes[mask1 | mask2]
raw_recipes.head()

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
id,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
31490,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
59389,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
5289,apple a day milk shake,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['combine ingredients in blender', 'cover and ...",NaN,"['milk', 'vanilla ice cream', 'frozen apple ju...",4
70971,bananas 4 ice cream pie,180,102353,2003-09-10,"['weeknight', 'time-to-make', 'course', 'main-...","[4270.8, 254.0, 1306.0, 111.0, 127.0, 431.0, 2...",8,"['crumble cookies into a 9-inch pie plate , or...",NaN,"['chocolate sandwich style cookies', 'chocolat...",6


In [8]:
#raw_recipes = raw_recipes.sample(100000)
raw_recipes.shape

(172596, 11)

<h3> Feature selection and conversion </h3>
Drop unneccessary features and convert nurtition column which is a list of calories, protiens, etc., into seperate columns each for calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates

In [9]:
raw_recipes = raw_recipes.drop(columns=['name','minutes', 'contributor_id', 'submitted','steps','description'])
raw_recipes.head(3)

,tags,nutrition,n_steps,ingredients,n_ingredients
id,,,,,
137739,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['winter squash', 'mexican seasoning', 'mixed ...",7
31490,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['prepared pizza crust', 'sausage patty', 'egg...",6
59389,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,"['spreadable cheese with garlic and herbs', 'n...",11


In [10]:
raw_recipes[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']] = raw_recipes.nutrition.str.split(",",expand=True) 
raw_recipes['calories'] = raw_recipes['calories'].apply(lambda x: x.replace("[" ,""))
raw_recipes['carbohydrates'] = raw_recipes['carbohydrates'].apply(lambda x: x.replace("]" ,""))
raw_recipes[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']] =  raw_recipes[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']].astype(float)

In [11]:
raw_recipes.head()

,tags,nutrition,n_steps,ingredients,n_ingredients,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates
id,,,,,,,,,,,,
137739,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
31490,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
59389,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,"['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
5289,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['milk', 'vanilla ice cream', 'frozen apple ju...",4,160.2,10.0,55.0,3.0,9.0,20.0,7.0
70971,"['weeknight', 'time-to-make', 'course', 'main-...","[4270.8, 254.0, 1306.0, 111.0, 127.0, 431.0, 2...",8,"['chocolate sandwich style cookies', 'chocolat...",6,4270.8,254.0,1306.0,111.0,127.0,431.0,220.0


In [12]:
raw_recipes = raw_recipes.drop(columns=['tags','nutrition', 'n_steps', 'ingredients','n_ingredients'])
raw_recipes.head()

,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates
id,,,,,,,
137739,51.5,0.0,13.0,0.0,2.0,0.0,4.0
31490,173.4,18.0,0.0,17.0,22.0,35.0,1.0
59389,368.1,17.0,10.0,2.0,14.0,8.0,20.0
5289,160.2,10.0,55.0,3.0,9.0,20.0,7.0
70971,4270.8,254.0,1306.0,111.0,127.0,431.0,220.0


<h3> 4. Data Normalization </h3>

In [13]:
raw_recipes_norm=pd.DataFrame(normalize(raw_recipes, axis=0))
raw_recipes_norm.columns = raw_recipes.columns
raw_recipes_norm.index = raw_recipes.index
raw_recipes_norm.head()

,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates
id,,,,,,,
137739,0.000092,0.000000,0.000034,0.000000,0.000076,0.000000,0.000103
31490,0.000311,0.000598,0.000000,0.000394,0.000832,0.000851,0.000026
59389,0.000661,0.000564,0.000026,0.000046,0.000529,0.000195,0.000515
5289,0.000288,0.000332,0.000145,0.000070,0.000340,0.000486,0.000180
70971,0.007668,0.008433,0.003450,0.002576,0.004803,0.010482,0.005665


<h3> Make predictions </h3><br/>
<ol>
<li>Find the top-rated recipes by the user.</li>
<li>Find the cosine distance of all recipes rated by user from recipe X.</li>
<li> Find the top-N nearest recipes to recipe X </li>
<li>Calculate the weighted average of the recipes to compute the predicted rating of recipe X</li>
   </ol>

In [14]:
def nutrition_recommender_rating(user_id, recipe_id, N):
    start = time()
    allRecipes = pd.DataFrame(raw_recipes_norm.index)
    allRecipes = allRecipes[allRecipes.id != recipe_id]
    allRecipes["distance"] = allRecipes["id"].apply(lambda x: cosine(raw_recipes_norm.loc[recipe_id], raw_recipes_norm.loc[x]))
    
    merged_df_test=  pd.merge(interactions_train, allRecipes,  how='inner',on='id')
    TopNRecommendation=merged_df_test[merged_df_test['user_id']==user_id].sort_values(by=['distance']).head(N)
    #print(TopNRecommendation.head(3))
    TopNRecommendation["rat_dis"]=TopNRecommendation['without_0_rating']*TopNRecommendation['distance']
    rating=round(TopNRecommendation['rat_dis'].sum()/TopNRecommendation['distance'].sum(),1)
    end = time()
    running_time = end - start
    #print(running_time)
    return rating

In [50]:
rating=nutrition_recommender_rating(8937,44551,10)
print(rating)

35.79746103286743
4.2


In [15]:
interactions_test=interactions_test.tail(100)
#print(interactions_test.shape)
user_list=[]
recipe_list=[]
act_rating=[]
est_ratings=[]
start = time()
for i, row in interactions_test.iterrows():
    u=row['user_id'];
    i=row['recipe_id'];
    actual_rating=row['without_0_rating']
    user_list.append(str(u))
    recipe_list.append(str(i))
    act_rating.append(actual_rating)
    pred=nutrition_recommender_rating(u,i,10)
    est_ratings.append(pred)
end = time()
running_time = end - start
print(running_time)

3636.212047815323


In [16]:
df_submission = pd.DataFrame(list(zip(user_list ,recipe_list,act_rating, est_ratings)), 
           columns =['user_id','recipe_id','Actual Rating', 'Predicted Rating'])

In [19]:
predictions1  = pd.read_csv('predictions_Nutrition_F100.csv')# read csv into ratings_df dataframe
predictions2 = pd.read_csv('predictions_Nutrition_L100.csv')
frames=[predictions1,predictions2]
predictions = pd.concat(frames)
rms = mean_squared_error(predictions['Actual Rating'], predictions['Predicted Rating'], squared=False)
print(rms)

0.9550916186418976


In [ ]:
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(df_submission['Actual Rating'], df_submission['Predicted Rating'], squared=False)

In [ ]:
print(rms)

In [17]:
df_submission.to_csv("predictions_Nutrition_L100.csv", index=False)